In [1]:
from lxml import html
import requests
import re
import pandas as pd

In [2]:
def process(season, episode, stripped):
    filtered = [s for s in stripped if not s.startswith("/") and s != "More on Genius"]
    labeled = []
    for f in filtered:
        m = re.search('([A-Z ]+):(.*)', f)
        if m:
            labeled.append([m.group(1),m.group(2)])
        elif f.startswith("["):
            labeled.append(["SCENE", f])
        else:
            labeled.append([f])
    merged = []
    chunks = []
    cur = "DEFAULT"
    for l in labeled:
        if len(l) >1:
            merged.append([cur,chunks])
            chunks = []
            cur = l[0]
            chunks.append(l[1])
        else:
            chunks.append(l[0])

    return [[season, episode, m[0], " ".join(m[1]).strip()] for m in merged]



In [ ]:
# these were in bad shape:
#  "https://genius.com/albums/Game-of-thrones/Season-7-scripts"


seasons = [
    "https://genius.com/albums/Game-of-thrones/Season-1-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-2-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-3-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-4-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-5-scripts",
    "https://genius.com/albums/Game-of-thrones/Season-6-scripts",
]

episodes = [];
for sidx, url in enumerate(seasons):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    ep_urls = tree.xpath("//div[contains(@class, 'chart_row')]/a/@href")    
    for eidx, ep in enumerate(ep_urls):
        if "needs-editing" in ep:
            print("skipping: ", ep)
            continue
        print(sidx, eidx, ep)
        page = requests.get(ep)
        tree = html.fromstring(page.content)
        raw = tree.xpath("//div[contains(@class, 'lyrics')]/descendant::*/text()")
        stripped = [r.strip() for r in raw if r.strip() != '']
        processed = process(sidx, eidx, stripped)
        episodes = episodes + processed
    

0 0 https://genius.com/Game-of-thrones-winter-is-coming-annotated
0 1 https://genius.com/Game-of-thrones-the-kingsroad-annotated
0 2 https://genius.com/Game-of-thrones-lord-snow-annotated
0 3 https://genius.com/Game-of-thrones-cripples-bastards-and-broken-things-annotated
0 4 https://genius.com/Game-of-thrones-the-wolf-and-the-lion-annotated
0 5 https://genius.com/Game-of-thrones-a-golden-crown-annotated
0 6 https://genius.com/Game-of-thrones-you-win-or-you-die-annotated
0 7 https://genius.com/Game-of-thrones-the-pointy-end-annotated
0 8 https://genius.com/Game-of-thrones-baelor-annotated
0 9 https://genius.com/Game-of-thrones-fire-and-blood-annotated
1 0 https://genius.com/Game-of-thrones-the-north-remembers-script-annotated
1 1 https://genius.com/Game-of-thrones-the-night-lands-script-annotated
1 2 https://genius.com/Game-of-thrones-what-is-dead-may-never-die-annotated
2 0 https://genius.com/Game-of-thrones-the-rains-of-castamere-annotated
skipping:  https://genius.com/Game-of-throne

In [ ]:
dat = pd.DataFrame(episodes)
dat.columns = ["season_idx", "episode_idx", "character", "utterance"]
dat.to_csv("got.csv")

In [ ]:
pd.options.display.max_colwidth = 1000
dat